# Exploring pyarrow and SQlite

In [1]:
import adbc_driver_manager.dbapi
import adbc_driver_sqlite.dbapi
import pyarrow as pa

from typing import Any, Callable

In [2]:
ARROW_CONSECUTIVE_DATABASE_FILENAME = "databases/arrow_consecutive.sqlite"
ARROW_CONCATENATED_DATABASE_FILENAME = "databases/arrow_concatenated.sqlite"
SQLITE_DATABASE_FILENAME = "databases/direct_sqlite.sqlite"

In [3]:
def execute_on_sqlite(database_uri: str, my_function: Callable, *args: Any, **kwargs: Any) -> Any:
    """Execute a function `my_function` on an SQlite database at `database_uri`.
    
    Returns:
        Result of `my_function`.
    """
    with adbc_driver_sqlite.dbapi.connect(database_uri) as connection:
        with connection.cursor() as cursor:
            result = my_function(cursor, *args, **kwargs)
        connection.commit()
        return result

In [4]:
def insert_data_into_table(cursor: adbc_driver_manager.dbapi.Cursor,
                           table_name: str,
                           data: pa.lib.Table,
                           mode: str) -> int:
    """Create a table and fill it with data or append data to an existing one.
    
    Args:
        cursor: database cursor
        table_name: name of the table to create or append to
        data: pa table
        mode: "create" a new table, or "append" to an existing one

    Returns:
        Number of rows inserted
    """

    if mode == "create":
        cursor.execute(f"DROP TABLE IF EXISTS {table_name}")

    result = cursor.adbc_ingest(table_name, data, mode=mode)
    return result

In [5]:
schema = pa.schema(
    [
        pa.field(name="name", type=pa.string()), # argument names optional
        ("age", pa.int8())                       # implicit conversion to pa.field

        # N.B. bool_ works in schema, but fails with the sqlite driver with:
        # NotSupportedError: ADBC_STATUS_NOT_IMPLEMENTED (2): Column 2 has unsupported type bool
    ]
)

In [6]:
data_1 = pa.table(
    [
        ["Alice", "Bob", "Carol"],
        [42, 46, 11],
    ],
    schema=schema,
)

data_2 = pa.table(
    [
        ["Dave", "Erin"],
        [73, 49],
    ],
    schema=schema,
)

## Create and write pyarrow table to SQlite, append more data

In [7]:
rows_created = execute_on_sqlite(f"file:{ARROW_CONSECUTIVE_DATABASE_FILENAME}", insert_data_into_table, "people", data_1, mode="create")
rows_appended = execute_on_sqlite(f"file:{ARROW_CONSECUTIVE_DATABASE_FILENAME}", insert_data_into_table, "people", data_2, mode="append")

print(f"{rows_created} rows created, {rows_appended} rows appended")

3 rows created, 2 rows appended


## "Concatenate" two pyarrow tables into a new one, write to SQlite
This will create a new view without overhead. Writing this concatenated view to SQlite will produce the same file as the above commands.

In [8]:
concatenated_data = pa.concat_tables([data_1, data_2])
concatenated_rows_created = execute_on_sqlite(f"file:{ARROW_CONCATENATED_DATABASE_FILENAME}", insert_data_into_table, "people", concatenated_data, mode="create")

print(f"{concatenated_rows_created} rows created")

5 rows created


## Create the same type of database directly in SQlite

In [9]:
sqlite_command = "DROP TABLE IF EXISTS people;" +\
"CREATE TABLE people (name, age);" + \
"INSERT INTO people (name, age) VALUES ('Alice', 42);" + \
"INSERT INTO people (name, age) VALUES ('Bob', 46);" + \
"INSERT INTO people (name, age) VALUES ('Carol', 11);" + \
"INSERT INTO people (name, age) VALUES ('Dave', 73);" + \
"INSERT INTO people (name, age) VALUES ('Erin', 49);"

!sqlite3 "{SQLITE_DATABASE_FILENAME}" "{sqlite_command}" ".exit"

## Comparing all SQlite files
Witness an empty return, meaning the databases are identical

In [10]:
!sqldiff databases/arrow_concatenated.sqlite databases/arrow_consecutive.sqlite
!sqldiff databases/arrow_concatenated.sqlite databases/direct_sqlite.sqlite